# Acquire the Data

## Acquiring Telco_churn data from SQL Database

In [14]:
import env
import pandas as pd
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from os import path
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import sklearn.preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import math
import split_scale

In [15]:
def get_db_url(db):
    return f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{db}'

In [16]:
def get_telco_data_from_mysql():
   
    # Use a double "%" in order to escape %'s default string formatting behavior.
    query = ''' Select * from customers
JOIN contract_types USING (contract_type_id)
Join internet_service_types USING (internet_service_type_id)
JOIN payment_types USING (payment_type_id);

    '''

    url = get_db_url("telco_churn") 
    df = pd.read_sql(query, url)
    return df

In [17]:
df = get_telco_data_from_mysql()

# Look through the data and drop any nulls

## Checking the data for unique values and possible future problems

In [18]:
df.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

## It appears that there are no null values in any of the columns, which might make things easier. Let's have a peek at the types of data we're working with.

In [19]:
df.columns

Index(['payment_type_id', 'internet_service_type_id', 'contract_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type'],
      dtype='object')

In [20]:
df.head()
df['online_security']

0                        No
1                        No
2                       Yes
3                        No
4                       Yes
5                       Yes
6                       Yes
7                        No
8                        No
9                       Yes
10                       No
11                      Yes
12                      Yes
13                      Yes
14                      Yes
15                       No
16                      Yes
17                       No
18                       No
19                       No
20                       No
21                       No
22                       No
23                      Yes
24                       No
25                       No
26                      Yes
27                      Yes
28                       No
29                       No
               ...         
7013    No internet service
7014    No internet service
7015    No internet service
7016    No internet service
7017    No internet 

In [21]:
new_df = df[['customer_id', 'churn', 'contract_type_id', 'contract_type', 'monthly_charges', 'total_charges', 'gender', 'partner', 'dependents', 'senior_citizen', 'phone_service', 'multiple_lines', 'internet_service_type', 'internet_service_type_id' ,'tech_support', 'streaming_tv', 'streaming_movies', 'online_security', 'online_backup', 'device_protection', 'payment_type', 'payment_type_id', 'paperless_billing']]

In [22]:
new_df.head()

,customer_id,churn,contract_type_id,contract_type,monthly_charges,total_charges,gender,partner,dependents,senior_citizen,...,internet_service_type_id,tech_support,streaming_tv,streaming_movies,online_security,online_backup,device_protection,payment_type,payment_type_id,paperless_billing
0,0003-MKNFE,No,1,Month-to-month,59.9,542.4,Male,No,No,0,...,1,No,No,Yes,No,No,No,Mailed check,2,No
1,0013-MHZWF,No,1,Month-to-month,69.4,571.45,Female,No,Yes,0,...,1,Yes,Yes,Yes,No,No,No,Credit card (automatic),4,Yes
2,0015-UOCOJ,No,1,Month-to-month,48.2,340.35,Female,No,No,1,...,1,No,No,No,Yes,No,No,Electronic check,1,Yes
3,0023-HGHWL,Yes,1,Month-to-month,25.1,25.1,Male,No,No,1,...,1,No,No,No,No,No,No,Electronic check,1,Yes
4,0032-PGELS,Yes,1,Month-to-month,30.5,30.5,Female,Yes,Yes,0,...,1,No,No,No,Yes,No,No,Bank transfer (automatic),3,No


# While there are no nulls that appear, there could be issues with trying to convert the datatypes of some columns

In [23]:
new_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [24]:
df = new_df.dropna()

In [25]:
df.total_charges = df.total_charges.astype(float, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 23 columns):
customer_id                 7032 non-null object
churn                       7032 non-null object
contract_type_id            7032 non-null int64
contract_type               7032 non-null object
monthly_charges             7032 non-null float64
total_charges               7032 non-null float64
gender                      7032 non-null object
partner                     7032 non-null object
dependents                  7032 non-null object
senior_citizen              7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
internet_service_type       7032 non-null object
internet_service_type_id    7032 non-null int64
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
online_security             7032 non-null object
online_backu

# It appears that the regex expression has removed a few rows. It has allowed me to change a datatype from an object to a float

## I want to parse down some of these columns to make things a little easier on my cognitive overhead. What I desire is a column with numeric encoders that represent a set of circumstances that can be explained in the data dictionary. First, I want to examine the unique values of these columns to make sure that they either say yes or no. If not, I need to do some replacing. Note that there are several columns that have id types already, which helps.

In [26]:
df.phone_service.unique()

df.tech_support.replace('No internet service', 'No', inplace = True)
df.tech_support.unique()

array(['No', 'Yes'], dtype=object)

In [27]:
## Taking a change in an effort to speed things up. Replacing 'No internet service' across the entire dataframe with 'No'
## Hopefully this is not an Icarus moment for myself

df.replace('No internet service', 'No', inplace = True)



In [28]:
df.multiple_lines.unique()

## My first replace did not convert everything. I need to execute another one.

array(['Yes', 'No', 'No phone service'], dtype=object)

In [29]:
df.replace('No phone service', 'No', inplace=True)

In [30]:
df.multiple_lines.unique()

array(['Yes', 'No'], dtype=object)

In [31]:
# Tired of scrolling up

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 23 columns):
customer_id                 7032 non-null object
churn                       7032 non-null object
contract_type_id            7032 non-null int64
contract_type               7032 non-null object
monthly_charges             7032 non-null float64
total_charges               7032 non-null float64
gender                      7032 non-null object
partner                     7032 non-null object
dependents                  7032 non-null object
senior_citizen              7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
internet_service_type       7032 non-null object
internet_service_type_id    7032 non-null int64
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
online_security             7032 non-null object
online_backu

In [32]:
df.contract_type.unique() # Have to leave this with multiple values

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [33]:
df.partner.unique()

array(['No', 'Yes'], dtype=object)

In [34]:
df.dependents.unique()

array(['No', 'Yes'], dtype=object)

In [35]:
df.phone_service.unique()

array(['Yes', 'No'], dtype=object)

In [36]:
df.tech_support.unique()

array(['No', 'Yes'], dtype=object)

In [37]:
df.paperless_billing.unique()

array(['No', 'Yes'], dtype=object)

In [38]:
df.gender.unique()

array(['Male', 'Female'], dtype=object)

In [39]:
df.multiple_lines.unique()

array(['Yes', 'No'], dtype=object)

__It appears that I have my columns where I want them to be with their responses. The next step is to aggregate a few of these columns together by writing code the labels certain circumstances with a number. The first attempt will be with the column for male and female.__

In [40]:
df['e_gender'] = df['gender'].apply({"Male":1,'Female':0}.get)
df.head()

,customer_id,churn,contract_type_id,contract_type,monthly_charges,total_charges,gender,partner,dependents,senior_citizen,...,tech_support,streaming_tv,streaming_movies,online_security,online_backup,device_protection,payment_type,payment_type_id,paperless_billing,e_gender
0,0003-MKNFE,No,1,Month-to-month,59.9,542.40,Male,No,No,0,...,No,No,Yes,No,No,No,Mailed check,2,No,1
1,0013-MHZWF,No,1,Month-to-month,69.4,571.45,Female,No,Yes,0,...,Yes,Yes,Yes,No,No,No,Credit card (automatic),4,Yes,0
2,0015-UOCOJ,No,1,Month-to-month,48.2,340.35,Female,No,No,1,...,No,No,No,Yes,No,No,Electronic check,1,Yes,0
3,0023-HGHWL,Yes,1,Month-to-month,25.1,25.10,Male,No,No,1,...,No,No,No,No,No,No,Electronic check,1,Yes,1
4,0032-PGELS,Yes,1,Month-to-month,30.5,30.50,Female,Yes,Yes,0,...,No,No,No,Yes,No,No,Bank transfer (automatic),3,No,0


In [55]:
# Creating coding references that reduce the number of columns that I have without using the label encoder function from scikit.

#Family describes the family type based on partners and children

conditions_1 =[
    (df['partner']=='Yes')& (df['dependents']=='Yes'),
    (df['partner']=='Yes')& (df['dependents']=='No'),
    (df['partner']=='No')& (df['dependents']=='Yes'),
    (df['partner']=='No')& (df['dependents']=='No')]
choices_1 = [0,1,2,3]
df['family'] = np.select(conditions_1, choices_1)

#Phone services describes whether someone has a phone plan and whether or not they have multiple lines

conditions_2 =[
    (df['phone_service']=='Yes')& (df['multiple_lines']=='Yes'),
    (df['phone_service']=='Yes')& (df['multiple_lines']=='No'),
    (df['phone_service']=='No')& (df['multiple_lines']== 'No')]
choices_2 = [0,1,2]
df['phone_services'] = np.select(conditions_2, choices_2)

#Streaming services denotes what streaming services someone has.

conditions_3 =[
    (df['streaming_tv']=='Yes')& (df['streaming_movies']=='Yes'),
    (df['streaming_tv']=='Yes')& (df['streaming_movies']=='No'),
    (df['streaming_tv']=='No')& (df['streaming_movies']=='Yes'),
    (df['streaming_tv']=='No')& (df['streaming_movies']=='No')]
choices_3 = [0,1,2,3]
df['streaming_services'] = np.select(conditions_3, choices_3)

#Online_services describes what types of online services someone has.

conditions_4=[
    (df['online_security']=='Yes')& (df['online_backup']=='Yes'),
    (df['online_security']=='Yes')& (df['online_backup']=='No'),
    (df['online_security']=='No')& (df['online_backup']=='Yes'),
    (df['online_security']=='No')& (df['online_backup']=='No')]
choices_4 = [0,1,2,3]
df['online_services'] = np.select(conditions_4, choices_4)

#Tech_support as to whether someone has tech support

df['e_tech_support'] = df['tech_support'].apply({"Yes":1,'No':0}.get)

# I'm confident you're getting the sentiment of these

df['e_device_protection'] = df['device_protection'].apply({"Yes":1,'No':0}.get)

# Looking for eco allies with this function

df['e_paperless_billing'] = df['paperless_billing'].apply({"Yes":1,'No':0}.get)

# df.drop(columns= 'e_senior_citizen', inplace = True)
df.head()

,customer_id,churn,contract_type_id,contract_type,monthly_charges,total_charges,gender,partner,dependents,senior_citizen,...,payment_type_id,paperless_billing,e_gender,family,phone_services,streaming_services,online_services,e_tech_support,e_device_protection,e_paperless_billing
0,0003-MKNFE,No,1,Month-to-month,59.9,542.40,Male,No,No,0,...,2,No,1,3,0,2,3,0,0,0
1,0013-MHZWF,No,1,Month-to-month,69.4,571.45,Female,No,Yes,0,...,4,Yes,0,2,1,0,3,1,0,1
2,0015-UOCOJ,No,1,Month-to-month,48.2,340.35,Female,No,No,1,...,1,Yes,0,3,1,3,1,0,0,1
3,0023-HGHWL,Yes,1,Month-to-month,25.1,25.10,Male,No,No,1,...,1,Yes,1,3,2,3,3,0,0,1
4,0032-PGELS,Yes,1,Month-to-month,30.5,30.50,Female,Yes,Yes,0,...,3,No,0,0,2,3,1,0,0,0


In [56]:
df.columns

Index(['customer_id', 'churn', 'contract_type_id', 'contract_type',
       'monthly_charges', 'total_charges', 'gender', 'partner', 'dependents',
       'senior_citizen', 'phone_service', 'multiple_lines',
       'internet_service_type', 'internet_service_type_id', 'tech_support',
       'streaming_tv', 'streaming_movies', 'online_security', 'online_backup',
       'device_protection', 'payment_type', 'payment_type_id',
       'paperless_billing', 'e_gender', 'family', 'phone_services',
       'streaming_services', 'online_services', 'e_tech_support',
       'e_device_protection', 'e_paperless_billing'],
      dtype='object')

# I've created a big old dataeframe that has many columns. I want to simplify what I am looking for, so I am going to create a new dataframe comprising all of the encoded columns. I will also create a data dictionary in my README.md so I can keep track of my encoding.

In [57]:
encoded_df = df[['customer_id', 'churn','e_gender' ,'contract_type_id', 'contract_type',
       'monthly_charges', 'total_charges','senior_citizen','internet_service_type_id','family', 'phone_services',
       'streaming_services', 'online_services', 'e_tech_support',
       'e_device_protection', 'e_paperless_billing' ]]

In [58]:
encoded_df.head()

,customer_id,churn,e_gender,contract_type_id,contract_type,monthly_charges,total_charges,senior_citizen,internet_service_type_id,family,phone_services,streaming_services,online_services,e_tech_support,e_device_protection,e_paperless_billing
0,0003-MKNFE,No,1,1,Month-to-month,59.9,542.40,0,1,3,0,2,3,0,0,0
1,0013-MHZWF,No,0,1,Month-to-month,69.4,571.45,0,1,2,1,0,3,1,0,1
2,0015-UOCOJ,No,0,1,Month-to-month,48.2,340.35,1,1,3,1,3,1,0,0,1
3,0023-HGHWL,Yes,1,1,Month-to-month,25.1,25.10,1,1,3,2,3,3,0,0,1
4,0032-PGELS,Yes,0,1,Month-to-month,30.5,30.50,0,1,0,2,3,1,0,0,0


# Looks a lot cleaner at this stage. Going to create a data dictionary at this point so I can have a quick reference to my encoding.